In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

print(f'OpenCV version: {cv2.__version__}')

In [ ]:
def show_img(img):
    fig = plt.figure(figsize = (20, 18))
    ax = fig.add_subplot(111)
    ax.imshow(img)

In [ ]:
img = cv2.imread('./data/OpenCV-CheatSheet.jpeg')
print(type(img))
print(f'np.shape(img) = {np.shape(img)}')

plt.imshow(img)

In [ ]:
color = img[0,0]
print(f'color = {color}') 
# [54 49 40] = [B, G, R]

In [ ]:
cond1 = np.logical_and(img[:,:,0]==color[0], img[:,:,1]==color[1])
cond2 = np.logical_and(cond1, img[:,:,2]==color[2])
indicesX, indicesY = np.where(cond2)

output_img_1 = np.copy(img)
output_img_1[indicesX, indicesY, :] = [255, 255, 255]

output_img_1 = cv2.cvtColor(output_img_1, cv2.COLOR_BGR2RGB) # OpenCV loaded image in BGR space but matplotlib requires RGB
plt.imshow(output_img_1)

show_img(output_img_1)

In [ ]:
# output_img_2 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# show_img(output_img_2)
color_hsv = cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_BGR2HSV)
color_hsv

# array([[[101,  66,  54]]], dtype=uint8)
# Hue = 101 and it determines the colour

In [ ]:
color_hsv[0, 0, 0]

In [ ]:
# Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255]. 
# to find nearest colours we can do +10 and -10
half_range_hue = 2
half_range_sat = 40
half_range_val = 40

color_hsv_lower = np.array([
    color_hsv[0, 0, 0] - half_range_hue, 
    color_hsv[0, 0, 1] - half_range_sat, 
    color_hsv[0, 0, 2] - half_range_val])
print(f'color_hsv_lower = {color_hsv_lower}')

color_hsv_upper = np.array([
    color_hsv[0, 0, 0] + half_range_hue, 
    color_hsv[0, 0, 1] + half_range_sat, 
    color_hsv[0, 0, 2] + half_range_val])
print(f'color_hsv_upper = {color_hsv_upper}')

img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# for 3D arrays cv2.inRange() returns a 2-D array which matches the size of the input image; the value of each 
mask_inverse = cv2.inRange(img_hsv, color_hsv_lower, color_hsv_upper)
print(f'np.shape(mask_inverse) = {np.shape(mask_inverse)}')
print(f'type(mask_inverse) = {type(mask_inverse)}')
print(f'type(mask_inverse[0][0]) = {type(mask_inverse[0][0])}')

# plt.imshow(cv2.cvtColor(mask_inverse, cv2.COLOR_GRAY2RGB))
show_img(cv2.cvtColor(mask_inverse, cv2.COLOR_GRAY2RGB))

mask = cv2.bitwise_not(mask_inverse)


# convert single channel mask back into 3 channels
mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)

# perform bitwise and on mask to obtain cut-out image that is not blue
img_masked = cv2.bitwise_and(img, mask_rgb)

# replace the cut-out parts with white
img_masked_replace_white = cv2.addWeighted(img_masked, 1, \
                                       cv2.cvtColor(mask_inverse, cv2.COLOR_GRAY2RGB), 1, 0)

# plt.imshow(cv2.cvtColor(img_masked_replace_white, cv2.COLOR_BGR2RGB))
show_img(img_masked_replace_white)